## Labelling with Inset Indonesia Lexicon

In [1]:
import pandas as pd
import json
from collections import OrderedDict
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
# Fungsi untuk membaca dan menghapus duplikasi teks
def read_and_remove_duplicates(input_file, text_column):
    """Membaca CSV dan menghapus duplikasi teks berdasarkan kolom teks kustom"""
    df = pd.read_csv(input_file, encoding='utf-8')
    df[text_column] = list(OrderedDict.fromkeys(df[text_column]))  # Menghapus duplikasi teks
    return df

# Fungsi untuk memuat leksikon kustom dan memperbarui SentimentIntensityAnalyzer
def load_lexicons():
    """Memuat leksikon kustom dan memperbarui VADER SentimentIntensityAnalyzer"""
    sia1A, sia1B, sia2 = SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer()
    sia1A.lexicon.clear()
    sia1B.lexicon.clear()
    sia2.lexicon.clear()

    # Memuat leksikon InSet dan SentiWords
    with open('../leksikon/inset/_json_inset-neg.txt') as f1A, \
         open('../leksikon/inset/_json_inset-pos.txt') as f1B, \
         open('../leksikon/sentistrength_id/_json_sentiwords_id.txt') as f2:
        insetNeg = json.load(f1A)
        insetPos = json.load(f1B)
        senti = json.load(f2)
    
    # Update leksikon ke analyzer
    sia1A.lexicon.update(insetNeg)
    sia1B.lexicon.update(insetPos)
    sia2.lexicon.update(senti)

    return sia1A, sia1B, sia2

# Fungsi untuk menentukan apakah sentimen positif berdasarkan leksikon InSet
def is_positive_inset(tweet, sia1A, sia1B):
    """Mengembalikan True jika tweet memiliki sentimen positif"""
    return sia1A.polarity_scores(tweet)["compound"] + sia1B.polarity_scores(tweet)["compound"] > 0

# Fungsi untuk menentukan apakah sentimen positif berdasarkan SentiWords
def is_positive_senti(tweet, sia2):
    """Mengembalikan True jika tweet memiliki sentimen positif"""
    return sia2.polarity_scores(tweet)["compound"] > 0

# Fungsi untuk menerapkan label berdasarkan analisis sentimen
def apply_sentiment_label(df, sia1A, sia1B, sia2, text_column):
    """Menerapkan label sentimen ke DataFrame berdasarkan leksikon"""
    df['label_inset'] = df[text_column].apply(lambda tweet: 'pos' if is_positive_inset(tweet, sia1A, sia1B) else 'neg')
    df['label_senti'] = df[text_column].apply(lambda tweet: 'pos' if is_positive_senti(tweet, sia2) else 'neg')
    return df

# Fungsi utama untuk pipeline pemrosesan dengan kolom teks kustom
def process_text(input_csv, output_csv, text_column):
    """Pipeline utama untuk memproses teks dan memberikan label sentimen berdasarkan kolom teks kustom"""
    
    # Membaca teks dan menghapus duplikasi
    df = read_and_remove_duplicates(input_csv, text_column)
    
    # Memuat leksikon dan SentimentIntensityAnalyzer
    sia1A, sia1B, sia2 = load_lexicons()
    
    # Menerapkan label sentimen ke DataFrame
    df = apply_sentiment_label(df, sia1A, sia1B, sia2, text_column)
    
    # Menyimpan hasil ke CSV
    df.to_csv(output_csv, index=False)
    print(f"Hasil disimpan di {output_csv}")

In [6]:

# Path input dan output file CSV
input_csv = '05_hasil_stemming_indo.csv'
output_csv = '06_hasil_labeling.csv'

# Jalankan pipeline
process_text(input_csv, output_csv)
text_column = 'text_steamindo'  # Kolom teks yang akan digunakan (custom)

# Jalankan pipeline dengan kolom teks kustom
process_text(input_csv, output_csv, text_column)

TypeError: process_text() missing 1 required positional argument: 'text_column'